## AWS S3에서 Parquet파일 read해서 redis에 저장

In [8]:
REDIS_INFO = {
    "host": "redis-master.redis-farm.svc.cluster.local",
    "port": "6379",
}

In [9]:
from pydatafabric.vault_utils import get_secrets

aws_info = get_secrets(mount_point="datafabric",path="aws/credentials/datafabric")

In [10]:
env = "dev" 
bucket = f"emart-datafabric-{env}"
s3_path = f"s3a://{bucket}/bigquery-db/redis/test.snappy.parquet"

print(s3_path)

s3a://emart-datafabric-dev/bigquery-db/redis/test.snappy.parquet


In [11]:
from pydatafabric.gcp import get_spark
spark = get_spark()

spark.conf.set("fs.s3a.fast.upload.buffer", "disk")
spark.conf.set("fs.s3a.buffer.dir", "/tmp")
spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])

In [12]:
df = spark.read.parquet(s3_path)

In [13]:
df.show(10)

+-------------+--------+----------+------------------------------------+------------------------+----------------------------------+----------+----------------------------------+
|      prdt_cd|store_cd|order_date|                            sku_name|           link_sku_name|                           comment|score_text|                          keywords|
+-------------+--------+----------+------------------------------------+------------------------+----------------------------------+----------+----------------------------------+
|1102150000000|    1018|  20220102|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살|    기름기가 많아서 
먹기 불편했음|       Bad|       기름기 많아서 먹기 불편했음|
|1102150000000|    1108|  20220101|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살| 기름이 너무너무 많아요.버린 기...|       Bad|          기름 많아요 기름 무게 만|
|1102150000000|    1108|  20220101|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살|세일전주에 샀을때 손질해보니 기...|       Bad|   전주 때 손질 기름 근육 고기 ...|
|1102150000000|    1048|  20220101|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살| 고기가 상태가 별

In [14]:
import redis
import json

# 레디스 연결
rd = redis.StrictRedis(host=REDIS_INFO["host"], port=REDIS_INFO["port"], db=0)

if df.count() > 0:
    json_df = df.toJSON().collect()[0]
    rd.set("test_eapp_review_keywords", json_df)

In [15]:
result = rd.get("test_eapp_review_keywords")
result = result.decode('utf-8')
result = dict(json.loads(result))

print(result)

{'prdt_cd': '1102150000000', 'store_cd': '1018', 'order_date': '20220102', 'sku_name': '(후레쉬팩)미국냉장초이스척아이로스트', 'link_sku_name': '(후레쉬팩)미국산CH갈비살', 'comment': '기름기가 많아서 \n먹기 불편했음', 'score_text': 'Bad', 'keywords': '기름기 많아서 먹기 불편했음'}


In [16]:
spark.stop()